In [ ]:
%matplotlib notebook
# General libraries
import numpy as np                # to deal with arrays, vectors, matrices...
import matplotlib.pyplot as plt   # to plot the data

# Tensorflow
import os
HOME = os.getenv('HOME')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # to get rid of the TF compilation warnings
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras.layers import Dense

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = 5,3
mpl.rcParams['font.size'] = 12.0

## Get/Prepare the data
Let us create a training and testing datasets.  
The data is created randomly so the testing and training datasets do not intersect  
The data will be normalized to the [0,1] interval

In [ ]:
import data  # hand-made library with a couple of functions
# Create the data
IN_train, OUT_train = data.gen_data_cool(300, norm=True)
IN_test,  OUT_test  = data.gen_data_cool(100, norm=True)

# Visualize the data
fig, ax = plt.subplots()
ax.scatter(IN_train, OUT_train, label='training')
ax.scatter(IN_test, OUT_test, label='testing')
ax.legend()
plt.show()

## Build the model
Let us create a model with two hidden layers formed by 2 and 5 sigmoid neurons respectively. It would look something like this:

<img src="test.svg" />

The output of the NN can be calculated as follows:
$$\text{Output} = \vec{a}_3 = \sigma\left( W_3\cdot\vec{a}_2 +b_3 \right)$$
where $\vec{a}_2$ is the output (activation) of the second hidden layer:
$$\vec{a}_2 = \sigma\left( W_2\cdot\vec{a}_1 +\vec{b}_2 \right)$$
where $\vec{a}_1$ is the output of the first hidden layer:
$$\vec{a}_1 = \sigma\left( W_1\cdot\vec{v}_{\text{input}} +\vec{b}_1 \right)$$

The NN, then, will be defined by three weight matrices ($W_1$, $W_2$ and $W_2$ in the figure) and three biases arrays (one for each layer) with shapes:
$$W_1\rightarrow(1,2) \qquad W_2\rightarrow(2,5) \qquad W_3\rightarrow(5,1)$$
$$\vec{b}_1\rightarrow(1,2) \qquad \vec{b}_2\rightarrow(1,5) \qquad \vec{b}_3\rightarrow(1,1)$$

In [ ]:
model = models.Sequential()
# Option 1
model.add( Dense(2, activation=None, input_shape=(1,)) )
model.add( Dense(5, activation=tf.nn.sigmoid) )
model.add( Dense(1, activation=None) )

# # Option 1
# model.add( Dense(3, activation=None, input_shape=(1,)) )
# model.add( Dense(10, activation='tanh') )
# model.add( Dense(10, activation='tanh') )
# model.add( Dense(1, activation=None) )


model.summary()

In [ ]:
model.compile(optimizer = 'adam',
              loss = 'mean_squared_error',
              metrics = ['accuracy'])

## Training

In [ ]:
# Training
from time import time
told = time()
history = model.fit(IN_train, OUT_train, epochs=100,
                    validation_data = (IN_test,OUT_test),
                    verbose=1)
print(f'\nTraining done in: {time()-told:.3f}s')

In [ ]:
# plot learning curve
err = history.history['loss']
acc = history.history['accuracy']

fig, ax = plt.subplots()
ax.plot(err,label='loss',lw=2)
ax.plot(acc,label='acc',lw=2)

ax.set_title('Learning curve')
ax.legend()
plt.show()

## Check on new data

In [ ]:
# Prediction over the whole domain
mx, Mx = np.min(IN_train), np.max(IN_train)
x_predict = np.linspace(mx, Mx,500)
x_predict = data.lineal_norm(x_predict)

y_predict = model.predict(x_predict)
#y_predict = data.lineal_norm(y_predict)   # should be unnecessary


fig, ax = plt.subplots()
ax.scatter(IN_train, OUT_train, label='train')
ax.scatter(IN_test,  OUT_test,label='test')
ax.plot(x_predict, y_predict,'k',lw=2, label='prediction')
ax.legend()
ax.set_title('Results')
plt.show()

In [ ]:
# # Save result
# model.save('my_first_NN.h5')